===============
Adaptado para Naive-Bayes
===============

In [ ]:
from google.colab import drive
import pandas as pd
# Montar Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
ruta="/content/drive/MyDrive/Chatbot/faq_felicidad_expandido_v2_con_nuevas_respuestas - faq_felicidad_expandido_v2_con_nuevas_respuestas.csv.csv"
df = pd.read_csv(ruta)
df

,pregunta,categoria,respuesta_compuesta,Unnamed: 3,nueva_respuesta
0,¿Qué es el índice de felicidad mundial?,definiciones,El Índice de Felicidad Mundial es una medida e...,NaN,La respuesta se relaciona con factores como el...
1,¿Qué significa el ranking de felicidad?,definiciones,El ranking de felicidad es la lista ordenada d...,NaN,Los informes internacionales utilizan encuesta...
2,¿Qué mide exactamente el índice de felicidad?,definiciones,El indice de la felicidad mide el bienestar su...,NaN,El propósito de este indicador es comprender c...
3,¿Qué entienden por felicidad en este informe?,definiciones,"El informe sentiende la felicidad como ""eval...",NaN,Los informes internacionales utilizan encuesta...
4,¿Qué diferencia hay entre el índice de felicid...,definiciones,"La diferencia que hay entre estos dos, es que...",NaN,La respuesta se relaciona con factores como el...
...,...,...,...,...,...
81,¿Por qué los países nórdicos suelen ser los má...,curiosidades,Los países nórdicos destacan por altos niveles...,NaN,Los informes internacionales utilizan encuesta...
82,¿Hay países pequeños que aparezcan en los prim...,curiosidades,"Sí, países como Islandia o Costa Rica, a pesar...",NaN,Diversas investigaciones en psicología positiv...
83,¿Puedes contarme un dato curioso sobre felicidad?,conversación,"Claro, por ejemplo: los estudios muestran que ...",NaN,El propósito de este indicador es comprender c...
84,¿Qué piensas de la felicidad?,conversación,Pienso que la felicidad es un objetivo univers...,NaN,La respuesta se relaciona con factores como el...


In [ ]:
!pip install sentence-transformers

In [ ]:
data = [
    # Categoría: saludo
    {"pregunta": "hola", "respuesta": "¡Hola! ¿En qué puedo ayudarte hoy?", "categoria": "saludo"},
    {"pregunta": "buenos días", "respuesta": "¡Buenos días! Estoy aquí para responder tus preguntas sobre el Informe Mundial de la Felicidad.", "categoria": "saludo"},
    {"pregunta": "buenas tardes", "respuesta": "¡Buenas tardes! ¿Te interesa saber algo sobre la felicidad global?", "categoria": "saludo"},
    {"pregunta": "buenas noches", "respuesta": "¡Buenas noches! Estoy disponible para ayudarte con información sobre el informe.", "categoria": "saludo"},

    # Categoría: contexto
    {"pregunta": "¿quién eres?", "respuesta": "Soy un chatbot diseñado para responder preguntas sobre el Informe Mundial de la Felicidad.", "categoria": "contexto"},
    {"pregunta": "¿qué eres?", "respuesta": "Soy una inteligencia artificial que te puede ayudar a entender mejor el Informe Mundial de la Felicidad.", "categoria": "contexto"},
    {"pregunta": "¿cuál es tu función?", "respuesta": "Mi función es ayudarte a explorar y comprender los datos y hallazgos del Informe Mundial de la Felicidad.", "categoria": "contexto"},

    # Categoría: pregunta
    {"pregunta": "¿qué es el informe mundial de la felicidad?", "respuesta": "Es un estudio anual que clasifica a los países según el nivel de felicidad de sus habitantes, basado en factores como salud, ingresos, libertad y apoyo social.", "categoria": "pregunta"},
    {"pregunta": "¿quién publica el informe mundial de la felicidad?", "respuesta": "El informe es publicado por la Red de Soluciones para el Desarrollo Sostenible de las Naciones Unidas.", "categoria": "pregunta"},
    {"pregunta": "¿cuál es el país más feliz del mundo?", "respuesta": "Según el último informe, Finlandia ha sido clasificado como el país más feliz del mundo.", "categoria": "pregunta"},
    {"pregunta": "¿qué factores se consideran en el informe?", "respuesta": "Se consideran factores como el PIB per cápita, apoyo social, esperanza de vida saludable, libertad para tomar decisiones, generosidad y percepción de corrupción.", "categoria": "pregunta"},
    {"pregunta": "¿cómo se mide la felicidad?", "respuesta": "La felicidad se mide a través de encuestas donde las personas califican su vida en una escala del 0 al 10, además de indicadores objetivos.", "categoria": "pregunta"},

    # Categoría: despedida
    {"pregunta": "gracias", "respuesta": "¡Con mucho gusto! ¿Hay algo más en lo que te pueda ayudar?", "categoria": "despedida"},
    {"pregunta": "eso es todo", "respuesta": "Fue un placer ayudarte. ¡Que tengas un excelente día!", "categoria": "despedida"},
    {"pregunta": "adiós", "respuesta": "Hasta luego. Si tienes más preguntas sobre la felicidad mundial, estaré aquí para ayudarte.", "categoria": "despedida"},
    {"pregunta": "nos vemos", "respuesta": "¡Nos vemos pronto! Recuerda que la felicidad también se comparte.", "categoria": "despedida"}
]


for _, row in df.iterrows():
    nueva_entrada = {
        "pregunta": row["pregunta"],
        "respuesta": row["respuesta_compuesta"],
        "categoria": row["categoria"]
    }
    data.append(nueva_entrada)

print(len(data))

from collections import Counter

conteo_por_categoria = Counter([d["categoria"] for d in data])
print(conteo_por_categoria)

102
Counter({'metodologia': 27, 'definiciones': 20, 'proposito': 15, 'conversación': 6, 'pregunta': 5, 'estadísticas': 5, 'consejos': 5, 'curiosidades': 5, 'saludo': 4, 'despedida': 4, 'contexto': 3, 'metodología': 3})


In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB

# Datos: pregunta → respuesta
X = [item["pregunta"] for item in data]
y = [item["respuesta"] for item in data]

# Creamos el pipeline: vectorizador + clasificador
chatbot_nb = Pipeline([
    ('tfidf', TfidfVectorizer(ngram_range=(1,2))),
    ('clf', MultinomialNB())
])

# Entrenamos el modelo
chatbot_nb.fit(X, y)

Pipeline(steps=[('tfidf', TfidfVectorizer(ngram_range=(1, 2))),
                ('clf', MultinomialNB())])

In [ ]:
vectorizador = chatbot_nb.named_steps['tfidf']
tokens = vectorizador.get_feature_names_out()
print(tokens[:50])  # muestra los primeros 50 tokens
print("Total de tokens:", len(tokens))

['2023' 'académicas' 'acceder' 'acceder los' 'actualiza' 'actualiza el'
 'además' 'además del' 'adicionales' 'adicionales cubre' 'adiós' 'al'
 'al final' 'al índice' 'algunos' 'algunos países' 'analiza' 'analiza la'
 'aparecen' 'aparecen al' 'aparecen en' 'aparezcan' 'aparezcan en' 'apoyo'
 'apoyo social' 'aumentar' 'aumentar la' 'ayudan' 'ayudan ser' 'año'
 'año en' 'años' 'bajo' 'bajo de' 'bienestar' 'bienestar subjetivo'
 'buena' 'buena posición' 'buenas' 'buenas noches' 'buenas tardes'
 'buenos' 'buenos días' 'cada' 'cada año' 'cada cuánto' 'calcula'
 'calcula el' 'cambió' 'cambió el']
Total de tokens: 582


In [ ]:
def responder_nb(pregunta_usuario):
    respuesta_predicha = chatbot_nb.predict([pregunta_usuario])[0]
    #print(pregunta_usuario)
    probas = chatbot_nb.predict_proba([pregunta_usuario])[0]
    confianza = max(probas)
    return respuesta_predicha, confianza


In [ ]:
while True:
    user_input = input("Tú: ")
    if user_input.lower() in ["salir", "exit", "adiós","chao", "adios", "bye","sayonara","hasta luego"]:
        print("Chatbot: ¡Hasta luego!")
        break

    respuesta, score = responder_nb(user_input)
    print(f"Chatbot ({score:.3f}): {respuesta}")


Tú: adios'
Chatbot (0.010):  El  informe sentiende la felicidad como "evaluaciones de la vida" hechas por los individuos sobre su satisfacción general con la misma, más que emociones momentáneas de placer o dolor.
Tú: salir
Chatbot: ¡Hasta luego!


In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

# Dividir los datos (80% entrenamiento, 20% prueba)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

y_pred = chatbot_nb.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))

Accuracy: 0.9523809523809523
